In [ ]:
from pdf2image import convert_from_path

from pdf2image.exceptions import (
 PDFInfoNotInstalledError,
 PDFPageCountError,
 PDFSyntaxError
)
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pytesseract import Output
import pdf2image
import pyautogui


In [ ]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract'
TESSDATA_PREFIX = 'C:/Program Files/Tesseract-OCR'

In [ ]:
path=r"C:\Users\swagatam\Downloads\sample.pdf"

In [ ]:
images = convert_from_path(path, 300,poppler_path=r'C:\Program Files (x86)\poppler-0.68.0_x86\poppler-0.68.0\bin')
for i, image in enumerate(images):
    fname = 'image'+str(i)+'.png'
    image.save(fname, "PNG")

In [ ]:
from pdf2image import convert_from_path

pages = convert_from_path(path, dpi=350,poppler_path=r'C:\Program Files (x86)\poppler-0.68.0_x86\poppler-0.68.0\bin')

image_name_list = []
count = 1
for page in pages:
    output_image_name = "Page_" + str(count) + ".png"
    page.save(output_image_name, 'PNG')
    image_name_list.append(output_image_name)
    count+=1

for image in image_name_list:
    image = cv2.imread(image)
    


In [ ]:
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
th=threshold_img
# display image
cv2.imshow('threshold image', threshold_img)
# Maintain output window until user presses a key
cv2.waitKey(0)
# Destroying present windows on screen
cv2.destroyAllWindows()
#configuring parameters for tesseract
custom_config = r'--oem 3 --psm 6'
# now feeding image to tesseract
details = pytesseract.image_to_data(threshold_img, output_type=Output.DICT, config=custom_config, lang='eng')
print(details.keys())

In [ ]:
total_boxes = len(details['text'])
for sequence_number in range(total_boxes):
    if int(details['conf'][sequence_number]) >30:
        (x, y, w, h) = (details['left'][sequence_number], details['top'][sequence_number], details['width'][sequence_number],  details['height'][sequence_number])
        threshold_img = cv2.rectangle(threshold_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [ ]:
parse_text = []
word_list = []
last_word = ''
for word in details['text']:
    if word!='':
           word_list.append(word)
           last_word = word
    if (last_word!='' and word == '') or (word==details['text'][-1]):
         parse_text.append(word_list)
         word_list = []

In [ ]:
file = open(r"C:\Users\swagatam\Downloads\out.txt", "w+")
#path_to_file=r"C:\Users\swagatam\Downloads\out.txt"
list_obj=parse_text
with open(path_to_file, "r+") as fileobj:
    # read all lines and only include lines that have something written
    lines = [x for x in fileobj.readlines() if x.strip()]
    # after reading reset the file position
    fileobj.seek(0)
    # iterate over the lines and words to add
    for line, word in zip(lines, list_obj):
        # create each new line with the added words
        new_line = "%s %s\n\n" % (line.rstrip(), word)
        # write the lines to the file
        fileobj.write(new_line)
file.close()